In [9]:
"""
🎯 학습 목표: Stuff Documents 체인과 ConversationBufferMemory를 결합한 RAG 파이프라인 구현
📚 핵심 개념: RAG(검색 증강 생성), 벡터 저장소, 임베딩, 메모리 시스템, LCEL 체인
🚀 실행 결과: 문서 컨텍스트를 기반으로 한 질의응답과 대화 기록 유지

=== 핵심 학습 포인트 ===
1. RAG 파이프라인: 문서 → 임베딩 → 벡터 저장 → 검색 → 생성
2. 메모리 통합: ConversationBufferMemory로 대화 기록 유지
3. LCEL 체인: RunnablePassthrough.assign으로 동적 메모리 주입
4. Stuff 체인: 모든 검색된 문서를 하나의 컨텍스트로 결합

체인에 다음 질문을 합니다:
    Aaronson은 유죄인가요?
    그가 테이블에 어떤 메시지를 썼나요?
    Julia는 누구인가요?
    
문서 출처: https://gist.github.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223
"""

# === 📦 Step 1: 필수 라이브러리 Import ===
# 🧠 개념: LangChain RAG 파이프라인 구현을 위한 핵심 컴포넌트들
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory

# === 🤖 Step 2: LLM 모델 초기화 ===
# 🧠 개념: ChatOpenAI는 OpenAI GPT 모델을 LangChain에서 사용할 수 있게 해주는 래퍼 클래스
# 📌 설정 의미:
#   - temperature=0.1: 낮은 값으로 일관된 답변 보장 (0.0=결정적, 1.0=창의적)
#   - RAG에서는 정확성이 중요하므로 낮은 temperature 사용
llm = ChatOpenAI(
    temperature=0.1,  # 📌 용도: 응답 일관성 제어, 타입: float, 범위: 0.0-2.0
)
# 💡 실무 팁: RAG 시스템에서는 hallucination 방지를 위해 낮은 temperature 권장

# === 💾 Step 3: 캐싱 시스템 구성 ===
# 🧠 개념: 임베딩 결과를 캐시하여 반복 실행 시 비용과 시간 절약
# 🔧 동작: 같은 텍스트의 임베딩을 재계산하지 않고 로컬 파일에서 로드
cache_dir = LocalFileStore("./.cache/")  # 📌 용도: 임베딩 캐시 저장소, 타입: LocalFileStore
# 💡 실무 팁: 프로덕션에서는 Redis 같은 분산 캐시 사용 권장

# === ✂️ Step 4: 텍스트 분할기 설정 ===
# 🧠 개념: 긴 문서를 LLM이 처리할 수 있는 크기의 청크로 분할
# 📌 설정 의미:
#   - tiktoken_encoder: OpenAI 토큰 계산 방식 사용 (정확한 토큰 수 계산)
#   - separator="\n": 줄바꿈 기준으로 자연스러운 분할
#   - chunk_size=600: 각 청크의 최대 토큰 수 (너무 크면 컨텍스트 오버플로우)
#   - chunk_overlap=100: 청크 간 겹침으로 문맥 연속성 유지
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",       # 📌 용도: 분할 기준, 타입: str
    chunk_size=600,       # 📌 용도: 청크 크기, 타입: int, 범위: 200-1000
    chunk_overlap=100,    # 📌 용도: 청크 겹침, 타입: int, 범위: 50-200
)
# 💡 실무 팁: chunk_size는 모델의 컨텍스트 창과 검색 효율성을 고려하여 조정

# === 📄 Step 5: 문서 로더 및 문서 분할 ===
# 🧠 개념: 외부 문서를 로드하고 검색 가능한 형태로 분할
# 🔧 동작: 파일 읽기 → 텍스트 추출 → 청크 분할 → Document 객체 생성
loader = UnstructuredFileLoader("../files/document.txt")  # 📌 용도: 문서 파일 로더, 타입: UnstructuredFileLoader
docs = loader.load_and_split(text_splitter=splitter)      # 📌 용도: 분할된 문서 리스트, 타입: List[Document]
# 💡 실무 팁: 다양한 파일 형식 지원을 위해 적절한 로더 선택 필요 (PDF, Word, Markdown 등)

# === 🔍 Step 6: 임베딩 및 벡터 저장소 구성 ===
# 🧠 개념: 텍스트를 벡터로 변환하여 의미적 검색을 가능하게 함
# 📌 임베딩: 텍스트의 의미를 고차원 벡터공간에 표현하는 기법
embeddings = OpenAIEmbeddings()  # 📌 용도: 텍스트 → 벡터 변환기, 타입: OpenAIEmbeddings

# 🔧 구현: 캐시와 임베딩 결합으로 성능 최적화
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings,    # 📌 기본 임베딩 모델
    cache_dir      # 📌 캐시 저장소
)
# 💡 실무 팁: 캐시 사용으로 임베딩 비용을 최대 90% 절약 가능

# 🧠 개념: FAISS는 Facebook에서 개발한 고성능 벡터 검색 라이브러리
# 🔧 동작: 벡터 간 유사도 계산으로 관련 문서 검색
vectorstore = FAISS.from_documents(docs, cached_embeddings)  # 📌 용도: 벡터 저장소, 타입: FAISS
# ⚠️ 주의: FAISS는 메모리 기반이므로 대규모 데이터에는 다른 벡터DB 고려

# === 🎯 Step 7: 검색기(Retriever) 생성 ===
# 🧠 개념: 사용자 질문과 유사한 문서 청크를 찾는 컴포넌트
# ✅ 수정: retriver → retriever (오타 수정)
retriever = vectorstore.as_retriever()  # 📌 용도: 벡터 검색기, 타입: VectorStoreRetriever
# 💡 실무 팁: search_kwargs={"k": 3}으로 검색 결과 수 제한 가능 (비용 절약)

# === 💬 Step 8: 메모리 시스템 초기화 ===
# 🧠 개념: ConversationBufferMemory는 전체 대화를 순서대로 저장하는 메모리 버퍼
# 📌 설정 의미:
#   - return_messages=True: HumanMessage/AIMessage 객체로 저장 (채팅 모델 호환)
#   - memory_key="chat_history": 프롬프트 템플릿의 변수명과 일치해야 함
memory = ConversationBufferMemory(
    return_messages=True,      # 📌 용도: 메시지 형태 반환, 타입: bool
    memory_key="chat_history"  # 📌 용도: 프롬프트 변수명, 타입: str
)
# 💡 실무 팁: 짧은 대화에는 BufferMemory, 긴 대화에는 SummaryBufferMemory 권장

# === 📝 Step 9: RAG 프롬프트 템플릿 구성 ===
# 🧠 개념: ChatPromptTemplate은 역할별 메시지를 구성하는 LangChain의 핵심 클래스
# 🔧 구조: system → memory → human 순서로 자연스러운 대화 흐름 구성
# ✅ 개선: 메모리 통합을 위해 MessagesPlaceholder 추가 및 한국어 프롬프트로 변경
prompt = ChatPromptTemplate.from_messages([
    (
        "system",  # 📌 역할: AI의 행동 방식 정의 (가장 중요!)
        """
        당신은 도움이 되는 AI 어시스턴트입니다. 주어진 컨텍스트만을 사용하여 질문에 답변해주세요.
        컨텍스트에 없는 정보는 "모르겠습니다"라고 답변하고, 추측하지 마세요.
        이전 대화 내용을 기억하고 일관성 있는 답변을 제공하세요.
        
        컨텍스트:
        {context}
        """
    ),
    MessagesPlaceholder(variable_name="chat_history"),  # 📌 메모리: 대화 기록이 삽입될 위치
    ("human", "{question}")  # 📌 변수: 사용자 질문이 전달될 플레이스홀더
])
# 💡 실무 팁: system 메시지가 AI 행동의 80%를 결정하므로 신중하게 작성

# === 🔄 Step 10: 헬퍼 함수들 정의 ===
# 🧠 개념: LCEL 체인에서 사용할 헬퍼 함수들

def load_memory(_):
    """
    📋 기능: 메모리에서 대화 히스토리를 로드하는 헬퍼 함수
    📥 입력: 체인 입력 딕셔너리 (자동으로 전달, 여기서는 사용 안 함)
    📤 출력: 메모리에서 로드한 대화 기록 (List[BaseMessage])
    💡 사용 시나리오: 체인 실행 전 메모리 컨텍스트 주입
    🔗 관련 개념: LCEL 기반 메모리 관리
    """
    memory_vars = memory.load_memory_variables({})
    return memory_vars["chat_history"]  # 📌 올바른 메모리 키 사용

def format_docs(docs):
    """
    📋 기능: Document 객체들을 하나의 문자열로 포맷
    📥 입력: Document 객체 리스트 (List[Document])
    📤 출력: 문서 내용이 결합된 문자열 (str)
    💡 사용 시나리오: 검색된 문서들을 프롬프트에 사용할 수 있는 형태로 변환
    🔗 관련 개념: Stuff Documents Chain - 모든 문서를 하나로 결합
    
    ⚠️ 핵심 수정: 이 함수가 TypeError 해결의 핵심!
    검색기가 반환하는 Document 객체를 문자열로 변환하여 tiktoken 에러 방지
    """
    if not docs:  # 📌 빈 리스트 처리
        return "관련 문서를 찾을 수 없습니다."
    
    # Document 객체에서 page_content 추출하여 문자열로 결합
    formatted_content = "\n\n".join([doc.page_content for doc in docs if hasattr(doc, 'page_content')])
    return formatted_content if formatted_content else "문서 내용을 읽을 수 없습니다."

# === ⚡ Step 11: LCEL 체인 구성 (수정된 버전) ===
# 🧠 개념: LangChain Expression Language로 컴포넌트들을 파이프(|)로 연결
# 🔧 동작 순서:
#   1. RunnablePassthrough.assign: 동적으로 메모리와 컨텍스트 주입
#   2. prompt: 프롬프트 템플릿에 변수들 적용
#   3. llm: 생성된 프롬프트를 LLM에 전달하여 응답 생성
# ✅ 핵심 수정: format_docs 함수 추가로 Document → String 변환
chain = (
    RunnablePassthrough.assign(
        # 📌 핵심: 동적 변수 주입 - 체인 실행 시마다 자동으로 호출됨
        chat_history=load_memory,                    # 메모리에서 대화 기록 로드
        context=lambda x: format_docs(retriever.get_relevant_documents(x["question"]))  # 🔧 수정: Document → String
    )
    | prompt  # 📌 단계: 프롬프트 템플릿 적용
    | llm     # 📌 단계: LLM 실행하여 최종 응답 생성
)
# 💡 실무 핵심: format_docs 함수로 Document 객체를 문자열로 변환하여 tiktoken 에러 해결

# === 🎭 Step 12: 대화 처리 함수 정의 ===
def chat_with_rag_memory(question: str) -> str:
    """
    📋 기능: RAG와 메모리가 통합된 대화 처리 시스템
    📥 입력: 사용자 질문 (str)
    📤 출력: AI 답변 (str)
    💡 사용 시나리오: 문서 기반 질의응답 + 대화 기록 유지
    🔗 관련 개념: RAG 파이프라인, 메모리 관리
    
    🔧 처리 과정:
    1. 체인 실행으로 응답 생성 (검색 + 포맷 + 메모리 + 생성)
    2. 새로운 대화를 메모리에 저장
    3. 응답 반환
    
    ⚠️ 에러 처리: try-catch로 안전한 실행 보장
    """
    try:
        # 체인 실행: 검색된 컨텍스트와 메모리를 활용하여 응답 생성
        response = chain.invoke({"question": question})  # 📌 LCEL 체인 실행
        
        # 📌 중요: 대화 후 메모리에 수동 저장 (LCEL에서는 자동 저장 안 됨)
        memory.save_context(
            {"input": question},           # 📌 사용자 입력
            {"output": response.content}   # 📌 AI 응답
        )
        
        return response.content
        
    except Exception as e:
        # 🚨 에러 처리: 사용자에게 친화적인 에러 메시지 반환
        error_msg = f"죄송합니다. 처리 중 오류가 발생했습니다: {str(e)}"
        print(f"❌ 오류 발생: {error_msg}")
        return error_msg

# === 🚀 Step 13: RAG 시스템 테스트 실행 ===
print("🔍 RAG + 메모리 시스템 테스트 시작")
print("=" * 50)

# 📌 테스트 시나리오: 1984 소설에 대한 연속적인 질문들
# 🎯 목표: 컨텍스트 검색 + 대화 기록 유지 검증

print("【질문 1】 Aaronson의 유죄 여부 확인")
a1 = chat_with_rag_memory("Aaronson은 유죄인가요?")
print(f"🤖 AI 답변: {a1}")
print()

print("【질문 2】 테이블 메시지 내용 (이전 답변과의 연관성 테스트)")  
a2 = chat_with_rag_memory("그가 테이블에 어떤 메시지를 썼나요?")
print(f"🤖 AI 답변: {a2}")
print()

print("【질문 3】 Julia 정체 확인")
a3 = chat_with_rag_memory("Julia는 누구인가요?")
print(f"🤖 AI 답변: {a3}")
print()

# === 📊 Step 14: 시스템 상태 분석 ===
print("📈 시스템 분석 결과")
print("=" * 50)

# 메모리 상태 확인
memory_content = memory.load_memory_variables({})["chat_history"]
print(f"💾 저장된 대화 수: {len(memory_content)}개 메시지")
print(f"📝 메모리 타입: {type(memory).__name__}")

# 벡터 저장소 정보
print(f"📚 로드된 문서 청크 수: {len(docs)}개")
print(f"🔍 벡터 저장소 타입: {type(vectorstore).__name__}")

print("\n✅ RAG + 메모리 통합 시스템 테스트 완료!")
print("\n🔗 다음 학습: Map Reduce 체인, 다른 메모리 타입 등으로 확장 가능")

# === 🔧 Step 15: 디버깅 정보 (선택사항) ===
print("\n🔧 디버깅 정보:")
print(f"📋 체인 타입: {type(chain)}")
print("📌 핵심 수정사항:")
print("   1. format_docs 함수 추가 - Document → String 변환")  
print("   2. 에러 처리 추가 - try-catch로 안전성 확보")
print("   3. 검색 로직 수정 - get_relevant_documents 직접 호출")
print("   4. 람다 함수 활용 - 동적 컨텍스트 생성")

🔍 RAG + 메모리 시스템 테스트 시작
【질문 1】 Aaronson의 유죄 여부 확인
🤖 AI 답변: 모르겠습니다.

【질문 2】 테이블 메시지 내용 (이전 답변과의 연관성 테스트)
🤖 AI 답변: 그가 쓴 메시지는 "FREEDOM IS SLAVERY"와 "TWO AND TWO MAKE FIVE" 그리고 "GOD IS POWER"입니다.

【질문 3】 Julia 정체 확인
🤖 AI 답변: Julia는 윈스턴과 함께 반란을 꾀하던 여성으로, 윈스턴이 사랑하는 상대입니다.

📈 시스템 분석 결과
💾 저장된 대화 수: 6개 메시지
📝 메모리 타입: ConversationBufferMemory
📚 로드된 문서 청크 수: 34개
🔍 벡터 저장소 타입: FAISS

✅ RAG + 메모리 통합 시스템 테스트 완료!

🔗 다음 학습: Map Reduce 체인, 다른 메모리 타입 등으로 확장 가능

🔧 디버깅 정보:
📋 체인 타입: <class 'langchain.schema.runnable.base.RunnableSequence'>
📌 핵심 수정사항:
   1. format_docs 함수 추가 - Document → String 변환
   2. 에러 처리 추가 - try-catch로 안전성 확보
   3. 검색 로직 수정 - get_relevant_documents 직접 호출
   4. 람다 함수 활용 - 동적 컨텍스트 생성


In [10]:
print("【질문 1】 Aaronson의 유죄 여부 확인")
a1 = chat_with_rag_memory("Aaronson은 유죄인가요?")
print(f"🤖 AI 답변: {a1}")
print()

【질문 1】 Aaronson의 유죄 여부 확인
🤖 AI 답변: 모르겠습니다.



In [12]:
print("Aaronson 유죄 여부 질문 횟수")
a1 = chat_with_rag_memory("Aaronson 유죄 여부 질문 횟수")
print(f"🤖 AI 답변: {a1}")
print()

Aaronson 유죄 여부 질문 횟수
🤖 AI 답변: Aaronson에 대한 유죄 여부에 대해 여러 번 물어주셨는데, 그 정보는 주어진 텍스트에는 나오지 않습니다. 그에 대한 정보가 없어서 답변을 제공할 수 없습니다.

